In [1]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,784 kB/s)
debconf: unable to initi

In [2]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 2.1 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [1]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

img = cv2.imread("bill.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

custom_config = r'-l eng --oem 1 --psm 6 '
d = pytesseract.image_to_data(thresh, config=custom_config, output_type=Output.DICT)
df = pd.DataFrame(d)

df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
for block in sorted_blocks:
    curr = df1[df1['block_num'] == block]
    sel = curr[curr.text.str.len() > 3]
    # sel = curr
    char_w = (sel.width / sel.text.str.len()).mean()
    prev_par, prev_line, prev_left = 0, 0, 0
    text = ''
    for ix, ln in curr.iterrows():
        # add new line when necessary
        if prev_par != ln['par_num']:
            text += '\n'
            prev_par = ln['par_num']
            prev_line = ln['line_num']
            prev_left = 0
        elif prev_line != ln['line_num']:
            text += '\n'
            prev_line = ln['line_num']
            prev_left = 0

        added = 0  # num of spaces that should be added
        if ln['left'] / char_w > prev_left + 1:
            added = int((ln['left']) / char_w) - prev_left
            text += ' ' * added
        text += ln['text'] + ' '
        prev_left += len(ln['text']) + added + 1
    text += '\n'
words_list = text.split('\n')
    #print(text)
print(words_list)


['', '                                                                                             bhttps//mrsinvoice.com ', '                                                                                                  Lp ', '                  I                                                                              | ', '        Your Company LLC Address 123, State, My Country P 111-222-333, F 111-222-334 ', '        BILL TO: ', '        P: 111-222-333, F: 111-222-334                          m                              . ', '        dlent@ccomplent ', '                                                          Contact Phone                  101-102-103 ', '        john Doe office                                   ayment  Terms                  ash on Delivery ', '        Office Road 38 ', '        P: 111-833-222, F: 122-222-334                            Amount    Due:    $4,170 ', '        office@example.net ', '          NO  PRODUCTS  / SERVICE                           

In [38]:
#words_list = text.split('\n')
#print(words_list)
#final_list = []
#for i in words_list:
#  final_list.append(i.strip(''))
#print(final_list)
final_output = []
for i in words_list:
    if i.strip() != '':
        final_output.append(i)
print(final_output)

['                                                                                             bhttps//mrsinvoice.com ', '                                                                                                  Lp ', '                  I                                                                              | ', '        Your Company LLC Address 123, State, My Country P 111-222-333, F 111-222-334 ', '        BILL TO: ', '        P: 111-222-333, F: 111-222-334                          m                              . ', '        dlent@ccomplent ', '                                                          Contact Phone                  101-102-103 ', '        john Doe office                                   ayment  Terms                  ash on Delivery ', '        Office Road 38 ', '        P: 111-833-222, F: 122-222-334                            Amount    Due:    $4,170 ', '        office@example.net ', '          NO  PRODUCTS  / SERVICE                               

In [39]:
k = []
for i in final_output:
    j = i.replace(' ','')
    k.append(j)
print(k)

['bhttps//mrsinvoice.com', 'Lp', 'I|', 'YourCompanyLLCAddress123,State,MyCountryP111-222-333,F111-222-334', 'BILLTO:', 'P:111-222-333,F:111-222-334m.', 'dlent@ccomplent', 'ContactPhone101-102-103', 'johnDoeofficeaymentTermsashonDelivery', 'OfficeRoad38', 'P:111-833-222,F:122-222-334AmountDue:$4,170', 'office@example.net', 'NOPRODUCTS/SERVICEQUANTITY/RATE/UNITAMOUNT', 'HOURS,PRICE', '1|tyre2$20$40', '2|SteeringWheet5$10$50', '3|Engineol40$15$150', '4|BrakePad2a$1000$2,400', 'Subtotal$275', 'Tax(10%)$275', 'GrandTotal$302.5', '‘THANKYOUFORYOURBUSINESS']
